# Sistema de Assistência Virtual



O objetivo desse código consiste em criar um assistente virtual com Processamento de Linguagem Natural (PLN), incluindo:

Text-to-Speech (TTS) → Transformar texto em áudio

Speech-to-Text (STT) → Transformar áudio (voz) em texto

Automação por comandos de voz → Executar ações como pesquisar no Wikipedia, abrir o YouTube e localizar farmácias próximas

### Módulo TTS (Texto para Áudio)

In [1]:
# Instalando a biblioteca gTTS

! pip install gTTS

In [17]:
# Importando a biblioteca gTTS

from gtts import gTTS

In [20]:
# Rodando o módulo com inputs do usuário para o texto e o idioma
text_to_say = input("Digite o texto para converter em fala: ")

# Escolher idioma (opcional - pt como padrão)
idiomas = {"1": "pt", "2": "en", "3": "es", "4": "fr"}
print("\nEscolha o idioma:\n1 - Português\n2 - English\n3 - Español\n4 - Français")
idioma_escolhido = input("Digite o número do idioma: ")

# Definir idioma (padrão: pt-BR)
language = idiomas.get(idioma_escolhido, "pt")

# Criar áudio
gtts_object = gTTS(text=text_to_say, lang=language, slow=False)
gtts_object.save("audio.mp3")

Digite o texto para converter em fala: Boa tarde

Escolha o idioma:
1 - Português
2 - English
3 - Español
4 - Français
Digite o número do idioma: 1


In [19]:
# Reproduzindo o áudio

from IPython.display import Audio

Audio("/content/audio.mp3")

### Módulo STT (Áudio para texto)

In [3]:
# Instalando a biblioteca Speech Recognition

!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 32.4 MB/s eta 0:00:00


In [12]:
# Utilizando JavaScript para capturar audio por meio do microfone do dispositivo

from google.colab import output
import base64
from IPython.display import Javascript, display, HTML

# Código JavaScript para capturar áudio com botões
js = Javascript("""
let mediaRecorder;
let audioChunks = [];

async function startRecording() {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    mediaRecorder = new MediaRecorder(stream);

    mediaRecorder.ondataavailable = event => {
        audioChunks.push(event.data);
    };

    mediaRecorder.onstop = async () => {
        const blob = new Blob(audioChunks, { type: 'audio/wav' });
        const reader = new FileReader();
        reader.readAsDataURL(blob);
        reader.onloadend = () => {
            google.colab.kernel.invokeFunction('notebook.save_audio', [reader.result], {});
        };
    };

    audioChunks = [];
    mediaRecorder.start();
    document.getElementById("status").innerText = "🎙 Gravando...";
}

function stopRecording() {
    mediaRecorder.stop();
    document.getElementById("status").innerText = "⏹ Gravação finalizada!";
}

// Criando botões no Colab
let container = document.createElement("div");
container.innerHTML = `
    <button onclick="startRecording()">🎙 Iniciar Gravação</button>
    <button onclick="stopRecording()">⏹ Parar Gravação</button>
    <p id="status">🔴 Aguardando...</p>
`;
document.body.appendChild(container);
""")

display(js)

# Função para salvar áudio no Colab
def save_audio(js_data):
    audio_bytes = base64.b64decode(js_data.split(',')[1])
    with open("audio.wav", "wb") as f:
        f.write(audio_bytes)
    print("✅ Áudio salvo como 'audio.wav'")

# Registrar a função para salvar o áudio
output.register_callback('notebook.save_audio', save_audio)



<IPython.core.display.Javascript object>

✅ Áudio salvo como 'audio.wav'


In [15]:
# Convertendo o formato capturado pelo JavaScript para WAV
!apt-get install -y ffmpeg
!ffmpeg -i audio.wav -ar 16000 -ac 1 -ab 192k -f wav audio_fixed.wav



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [16]:
import speech_recognition as sr

recognizer = sr.Recognizer()

# Carregar o arquivo de áudio corrigido
with sr.AudioFile("audio_fixed.wav") as source:
    print("🎧 Processando áudio...")
    audio = recognizer.record(source)

# Reconhecer o texto com Google Speech Recognition
try:
    texto = recognizer.recognize_google(audio, language="pt-BR")
    print("📝 Texto reconhecido:", texto)
except sr.UnknownValueError:
    print("❌ Não foi possível entender o áudio")
except sr.RequestError:
    print("❌ Erro ao conectar com o serviço de reconhecimento")


🎧 Processando áudio...
📝 Texto reconhecido: Bom dia


### Módulo Automação por comandos de voz

In [24]:
# Importando dependências

import webbrowser
import datetime
import os
from gtts import gTTS
from pygame import mixer
import base64
import speech_recognition as sr
from google.colab import output
import IPython.display as display
import ipywidgets as widgets

In [25]:
# Capturando áudio com microfone do dispositivo usando código Java

from google.colab import output
import base64
from IPython.display import Javascript, display

# Código JavaScript para capturar áudio com botões
js = Javascript("""
let mediaRecorder;
let audioChunks = [];

async function startRecording() {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    mediaRecorder = new MediaRecorder(stream);

    mediaRecorder.ondataavailable = event => {
        audioChunks.push(event.data);
    };

    mediaRecorder.onstop = async () => {
        const blob = new Blob(audioChunks, { type: 'audio/wav' });
        const reader = new FileReader();
        reader.readAsDataURL(blob);
        reader.onloadend = () => {
            google.colab.kernel.invokeFunction('notebook.save_voice_command', [reader.result], {});
        };
    };

    audioChunks = [];
    mediaRecorder.start();
    document.getElementById("status").innerText = "🎙 Gravando comando de voz...";
}

function stopRecording() {
    mediaRecorder.stop();
    document.getElementById("status").innerText = "⏹ Comando de voz capturado!";
}

// Criando botões no Colab
let container = document.createElement("div");
container.innerHTML = `
    <button onclick="startRecording()">🎙 Iniciar Comando de Voz</button>
    <button onclick="stopRecording()">⏹ Parar Comando de Voz</button>
    <p id="status">🔴 Aguardando comando...</p>
`;
document.body.appendChild(container);
""")

display(js)

# Função para salvar o comando de voz no Colab
def save_voice_command(js_data):
    audio_bytes = base64.b64decode(js_data.split(',')[1])
    with open("voice_command.wav", "wb") as f:
        f.write(audio_bytes)
    print("✅ Comando de voz salvo como 'voice_command.wav'")

# Registrar a função para salvar o comando de voz
output.register_callback('notebook.save_voice_command', save_voice_command)


<IPython.core.display.Javascript object>

✅ Comando de voz salvo como 'voice_command.wav'


In [28]:
# Convertendo o formato capturado pelo JavaScript para WAV
!apt-get install -y ffmpeg
!ffmpeg -i voice_command.wav -ar 16000 -ac 1 -ab 192k -f wav voice_command_fixed.wav

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq -

In [29]:
# Transformando o comando de voz em texto

import speech_recognition as sr

# Função para converter áudio em texto
def transcrever_comando():
    recognizer = sr.Recognizer()

    with sr.AudioFile("voice_command_fixed.wav") as source:
        print("🎧 Processando comando de voz...")
        audio = recognizer.record(source)

    try:
        comando_texto = recognizer.recognize_google(audio, language="pt-BR")
        print(f"🗣 Comando reconhecido: {comando_texto}")
        return comando_texto.lower()
    except sr.UnknownValueError:
        print("❌ Não foi possível entender o áudio.")
        return None
    except sr.RequestError:
        print("⚠️ Erro ao conectar ao serviço de reconhecimento de voz.")
        return None

# Rodar o reconhecimento
comando = transcrever_comando()


🎧 Processando comando de voz...
🗣 Comando reconhecido: Abrir o Google


In [34]:
# Processando o comando

import re

def processar_comando(comando):
    comando = comando.lower()  # Normaliza para minúsculas

    if "horas" in comando:
        agora = datetime.datetime.now().strftime("%H:%M")
        resposta = f"Agora são {agora}."

    elif re.search(r"\b(abrir|abra|abre|acesse|acessa|acessar|vá para|ir para)\b", comando):  # Mais flexível
        if "google" in comando:
            webbrowser.open("https://www.google.com")
            resposta = "Abrindo o Google."
        elif "youtube" in comando:
            webbrowser.open("https://www.youtube.com")
            resposta = "Abrindo o YouTube."
        else:
            resposta = "Não entendi qual site deseja abrir."

    elif "pesquise sobre" in comando:
        termo = comando.replace("pesquise sobre", "").strip()
        try:
            resumo = wikipedia.summary(termo, sentences=1, lang="pt")
            resposta = f"De acordo com a Wikipédia: {resumo}"
        except wikipedia.exceptions.DisambiguationError:
            resposta = f"Encontrei várias opções para {termo}. Seja mais específico."
        except wikipedia.exceptions.PageError:
            resposta = "Não encontrei informações sobre esse assunto."

    elif "piada" in comando:
        resposta = pyjokes.get_joke(language="pt")

    elif "sair" in comando:
        resposta = "Até mais! Encerrando o assistente."
        print(resposta)
        return resposta

    else:
        resposta = "Desculpe, não entendi o comando."

    print("🤖:", resposta)
    return resposta


In [35]:
# Rodando o processamento do comando reconhecido
if comando:
    resposta = processar_comando(comando)

🤖: Abrindo o Google.
